In [2]:
__author__ = 'biherrera'
__email__ = 'biherrera@uc.cl'

import sys
import os

# The following line adds the directory to the path in order to cross-reference
# files in the repo
sys.path.append(os.getcwd()[:-6])
print("Relative path: ", os.getcwd()[:-6])

import dolfin
from src.model import PerfusionGasExchangeModel
from src.params import params
    
print("Imported src files")
print("Starting...")
folder = "rve_40_12_it"
path = os.path.join("../local-results-data", folder)
model = PerfusionGasExchangeModel(folder_path=path, params=params, solver='mumps', f_dim = 1, vecf_dim=2)

max_dims = [39.9592, 40.0040, 39.9904]
min_dims = [0.02681, -0.01336, 0.03181]

print("Model initialised")
model.import_mesh(
    os.path.join("../raw-data/40_rve_12", "rve_40_12.xdmf"), type="xdmf", 
    periodic=False, max_dims=max_dims, min_dims=min_dims, tol=0.1
)
print("Mesh imported")
# model.mesh = dolfin.refine(model.mesh)
print("Mesh not refined")
print("Starting (P) simulation")
model.sim_p(save=True, meshtype="tkd")
print("(P) simulation done")


Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling
Imported src files
Starting...
Model initialised
Mesh imported
Coordinates have shape  (11318, 3)
Mesh imported
Mesh not refined
Starting (P) simulation

 ############## (P) problem simulation ############
Instancing boundaries
gamma_in.dir_min =  0.02681
gamma_in.dir_max =  39.9592
gamma_out.dir_min =  0.02681
gamma_out.dir_max =  39.9592
gamma_air.dir_min_y =  -0.01336
gamma_air.dir_max_y =  40.004
gamma_air.dir_min_z =  0.03181
gamma_air.dir_max_z =  39.9904
boundaries instanced
self.f_dim = 1
self.vecf_dim = 2
P problem solved
u problem solved
Saving solutions
saved
(P) simulation done


In [ ]:
print("Starting (T) simulation")
x = model.sim_t(hb=False, save=True, solver="bicgstab")
print("Finished 1st (linear) guess generation")
y = model.sim_t(hb=True, save=True, guess=x, solver="mumps", preconditioner="default", opt=4)
print("Finished 2nd (non-linear) guess generation")
solution = model.sim_t(hb=True, save=True, guess=y, solver="mumps", preconditioner="default")
print("Done")

Starting (T) simulation

 ############## (LT) and (T) problem simulation ############
began sim_t, with solver =  bicgstab  and WITHOUT preconditioner.
Solving with solver = bicgstab and without preconditioner.
Finished 1st (linear) guess generation

 ############## (LT) and (T) problem simulation ############
began sim_t, with solver =  mumps  and preconditioner =  default
Solving with solver = mumps and preconditioner = default.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
